In [20]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics.pairwise import linear_kernel
import nltk

In [21]:
# Cargo los datos de las películas
df_movies = pd.read_csv("../data/launch/movies.csv")
df_genres = pd.read_csv("../data/launch/genres.csv")
df_movie_genres = pd.read_csv("../data/launch/movie_genres.csv")

### Realizamos Join y Limpieza de datos

In [22]:
# Combinar los DataFrames df_movie_genres y df_genres para añadir nombres de géneros a cada película.
df_movie_genres_2 = pd.merge(df_movie_genres, df_genres[["genre_id", "name"]], on="genre_id", how="left")

# Paso 1: Agrupar los géneros por id_movie y combinarlos en listas
# Utilizar el método groupby para agrupar los géneros por el ID de la película y luego unir los nombres de los géneros en una lista.
grouped_genres = df_movie_genres_2.groupby("id")["name"].agg(" ".join).reset_index()

# Paso 2: Realizar el join entre df_movies y grouped_genres
# Combinar el DataFrame df_movies con el DataFrame grouped_genres utilizando el ID de la película como clave de combinación.
df_movies = pd.merge(df_movies, grouped_genres, on="id", how="left")

# Cambiar el nombre de la columna "name" a "genres_name"
# Renombrar la columna "name" del DataFrame df_movies a "genres_name" para hacerla más descriptiva.
df_movies = df_movies.rename(columns={"name": "genres_name"})

# Cambiar el tipo de columna a string para la columna "genres_name"
# Cambiar el tipo de datos de la columna "genres_name" del DataFrame df_movies a tipo "string".
df_movies = df_movies.astype({"genres_name": "string"})

# Reemplazar los valores faltantes con "No Genre" en la columna "genres_name"
# Rellenar los valores faltantes en la columna "genres_name" del DataFrame df_movies con el texto "No Genre".
df_movies = df_movies.fillna({"genres_name": "No Genre"})

# Reemplazar todas las instancias de "," con "" en la columna "overview"
# Utilizar el método str.replace() para eliminar todas las comas "," en la columna "overview" del DataFrame df_movies.
df_movies["overview"] = df_movies["overview"].str.replace(",", "", case=False, regex=False)


In [23]:
df_movies.head(1)

,id,title,original_language,overview,tagline,status,runtime,release_date,release_year,budget,revenue,return,popularity,vote_average,vote_count,collection_id,has_collection,genres_name
0,2,Ariel,fi,Taisto Kasurinen is a Finnish coal miner whose...,NaN,Released,69,1988-10-21,1988,0.0,0.0,0.0,3.86,7.1,44,NaN,0,Drama Crime


In [24]:
df_movies = df_movies[["overview", "popularity","title", "genres_name"]]

In [25]:
df_movies.head(1)

,overview,popularity,title,genres_name
0,Taisto Kasurinen is a Finnish coal miner whose...,3.86,Ariel,Drama Crime


In [26]:
df_ML = df_movies.copy()

In [27]:
df_ML.shape

(45349, 4)

In [28]:
df_ML = df_ML.dropna().reset_index(drop=True)

In [29]:
df_ML.shape

(44408, 4)

In [30]:
df_ML=df_ML.head(100)

# Prueba de Modelo de Recomendación ML TF-IDF # 1
### Primero se buscan titulos completamente similares, luego similitud de todos los generos, luego similitud de descripción y por último puntaje de popularidad

In [31]:
# Preprocesamiento de los títulos
df_ML["title_clean"] = df_ML["title"].apply(lambda x: x.lower())

# Crear una matriz de términos
stopwords_custom = nltk.corpus.stopwords.words("english")  # Agrega aquí stopwords personalizados
count_vectorizer = CountVectorizer(stop_words=stopwords_custom)
titulo_matrix = count_vectorizer.fit_transform(df_ML["title_clean"])

# Calcular la similitud de coseno entre títulos
titulo_similarity = cosine_similarity(titulo_matrix)

# Índice de la película de referencia
indice_pelicula_referencia = 0  # Cambia esto al índice de la película que deseas recomendar

# Obtener películas similares por título
similar_movies_by_title = list(enumerate(titulo_similarity[indice_pelicula_referencia]))
similar_movies_by_title = sorted(similar_movies_by_title, key=lambda x: x[1], reverse=True)
# Preprocesamiento de los géneros
df_ML["genres_clean"] = df_ML["genres_name"].apply(lambda x: x.lower().split())

# Calcular la similitud de Jaccard entre géneros
def jaccard_similarity(genre_list1, genre_list2):
    intersection = len(set(genre_list1).intersection(genre_list2))
    union = len(set(genre_list1).union(genre_list2))
    return intersection / union if union != 0 else 0.0

# Índice de la película de referencia
indice_pelicula_referencia = 0  # Cambia esto al índice de la película que deseas recomendar

# Obtener películas similares por género
similar_movies_by_genre = []
for i, row in df_ML.iterrows():
    similarity = jaccard_similarity(df_ML["genres_clean"][indice_pelicula_referencia], row["genres_clean"])
    similar_movies_by_genre.append((i, similarity))
similar_movies_by_genre = sorted(similar_movies_by_genre, key=lambda x: x[1], reverse=True)
# Preprocesamiento de las descripciones
df_ML["overview_clean"] = df_ML["overview"].apply(lambda x: x.lower())

# Crear matriz TF-IDF para descripciones
stopwords_custom = nltk.corpus.stopwords.words("english")
tfidf_vectorizer = TfidfVectorizer(stop_words=stopwords_custom)
descripcion_matrix = tfidf_vectorizer.fit_transform(df_ML["overview_clean"])

# Calcular la similitud de coseno entre descripciones
descripcion_similarity = cosine_similarity(descripcion_matrix)

# Índice de la película de referencia
indice_pelicula_referencia = 0  # Cambia esto al índice de la película que deseas recomendar

# Obtener películas similares por descripción
similar_movies_by_description = list(enumerate(descripcion_similarity[indice_pelicula_referencia]))
similar_movies_by_description = sorted(similar_movies_by_description, key=lambda x: x[1], reverse=True)
# Ordenar las películas por puntaje de popularidad (en orden descendente)
df_ML_sorted = df_ML.sort_values(by="popularity", ascending=False)

# Índice de la película de referencia
indice_pelicula_referencia = 0  # Cambia esto al índice de la película que deseas recomendar

# Obtener las 5 películas principales por puntaje de popularidad
top_popular_movies = df_ML_sorted.iloc[:5]



In [32]:
# Crear diccionario de indexación por título
title_to_index = {title: idx for idx, title in enumerate(df_ML["title"])}

In [33]:
def recommend_movies(title, df_ML, title_to_index):
    
    movie_index = title_to_index.get(title)
    if movie_index is None:
        return ["No se encontró la película en la base de datos."]
    
    # Matrices precalculadas
    stopwords_custom = nltk.corpus.stopwords.words("english")  # Agrega aquí stopwords personalizados
    count_vectorizer = CountVectorizer(stop_words=stopwords_custom)
    titulo_matrix = count_vectorizer.fit_transform(df_ML["title_clean"])
    titulo_similarity = cosine_similarity(titulo_matrix)
    similar_movies_title = list(enumerate(titulo_similarity[movie_index]))

    # Coincidencia de Género
    def jaccard_similarity(genre_list1, genre_list2):
        intersection = len(set(genre_list1).intersection(genre_list2))
        union = len(set(genre_list1).union(genre_list2))
        return intersection / union if union != 0 else 0.0

    similar_movies_genre = []
    for i, row in df_ML.iterrows():
        similarity = jaccard_similarity(df_ML["genres_clean"][movie_index], row["genres_clean"])
        similar_movies_genre.append((i, similarity))

    # Coincidencia de Descripción
    stopwords_custom = nltk.corpus.stopwords.words("english")  # Agrega aquí stopwords personalizados
    tfidf_vectorizer = TfidfVectorizer(stop_words=stopwords_custom)
    descripcion_matrix = tfidf_vectorizer.fit_transform(df_ML["overview_clean"])
    descripcion_similarity = cosine_similarity(descripcion_matrix)
    similar_movies_description = list(enumerate(descripcion_similarity[movie_index]))

    # Combinar y ordenar recomendaciones
    all_similarities = [(idx, (sim_title + sim_genre + sim_description) / 3.0)
                        for (idx, sim_title), (_, sim_genre), (_, sim_description)
                        in zip(similar_movies_title, similar_movies_genre, similar_movies_description)]
    all_similarities = sorted(all_similarities, key=lambda x: x[1], reverse=True)

    # Obtener películas recomendadas
    recommended_movies = []
    for idx, _ in all_similarities:
        if idx != movie_index and len(recommended_movies) < 5:
            recommended_movies.append(df_ML.iloc[idx]["title"])

    return recommended_movies

In [35]:
# Buscamos la Película de preferencia
recommended_movies = recommend_movies("Ariel", df_ML, title_to_index)
print(recommended_movies)

['Taxi Driver', 'Paradise Now', 'A History of Violence', 'Dancer in the Dark', 'Leon: The Professional']


# Creación de una función para el pregrabado general de las recomendaciones
### Este modelo consume muchos recursos de memoria por esto hemos optado por usar Microsoft Fabric quien nos da un poco más de recursos y obtener 15000 ejemplares de recomendaciones.

In [16]:
# Crear una lista para almacenar los datos
result_data = []

# Recorrer cada título en el DataFrame
for title in df_ML["title"]:
    recommended_movies = recommend_movies(title, df_ML, title_to_index)
    recommended_movies_str = ", ".join(recommended_movies)
    
    result_data.append({"movie_title": title, "recommended_titles": recommended_movies_str})

# Crear un DataFrame a partir de la lista de datos
result_df = pd.DataFrame(result_data, columns=["movie_title", "recommended_titles"])

# Guardar el DataFrame en un archivo CSV
result_df.to_csv("../data/ML/recomendaciones_peliculas_ML1_100.csv", index=False)

In [17]:
def get_recommendations_for_title(title):
    # Cargar el DataFrame precalculado desde el archivo CSV
    # Cargamos un DataFrame llamado "preview_recommendations" a partir del archivo CSV que contiene las recomendaciones precalculadas.
    preview_recommendations = pd.read_csv("../data/ML/recomendaciones_peliculas_ML1_100.csv")

    # Filtrar las recomendaciones para el título dado
    recommendations = preview_recommendations[preview_recommendations["movie_title"] == title]
    
    # Verificar si hay recomendaciones para el título dado
    if not recommendations.empty:
        # Si hay recomendaciones, obtener los títulos recomendados y dividirlos en una lista utilizando la coma como separador
        recommended_titles = recommendations.iloc[0]["recommended_titles"]
        return recommended_titles.split(", ")
    else:
        # Si no se encontraron recomendaciones para el título dado, devolver un mensaje de error
        return ["Película no encontrada en la Base de Datos."]

In [36]:
# Buscamos la Película de preferencia
title_to_search = "Ariel"
recommended_titles = get_recommendations_for_title(title_to_search)
print(recommended_titles)

['Taxi Driver', 'Paradise Now', 'A History of Violence', 'Dancer in the Dark', 'Leon: The Professional']
